In [ ]:
#Libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Add, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from itertools import product

In [1]:
# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 8

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    for category in categories:
        class_num = categories.index(category)
        clear_path = os.path.join(base_dir, subfolders[0], category)
        clouds_path = os.path.join(base_dir, subfolders[1], category)
        clear_images = sorted(os.listdir(clear_path))
        clouds_images = sorted(os.listdir(clouds_path))
        
        for clear_img_name, clouds_img_name in zip(clear_images, clouds_images):
            if clear_img_name.endswith('.png') and clouds_img_name.endswith('.png'):
                clear_img_path = os.path.join(clear_path, clear_img_name)
                clouds_img_path = os.path.join(clouds_path, clouds_img_name)
                
                clear_img = tf.keras.preprocessing.image.load_img(clear_img_path, target_size=(img_height, img_width))
                clouds_img = tf.keras.preprocessing.image.load_img(clouds_img_path, target_size=(img_height, img_width))
                
                clear_img_array = tf.keras.preprocessing.image.img_to_array(clear_img)
                clouds_img_array = tf.keras.preprocessing.image.img_to_array(clouds_img)
                
                combined_img = np.concatenate((clear_img_array, clouds_img_array), axis=-1)
                
                data.append(combined_img)
                labels.append(class_num)
    return np.array(data), np.array(labels)

data, labels = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)

# Normalize the images
data = data / 255.0

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")


TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (4832, 64, 64, 6)
Validation data shape: (1208, 64, 64, 6)
Training labels shape: (4832, 2)
Validation labels shape: (1208, 2)


# Retraining with best parameters found

In [2]:
# Define data augmentation
def create_datagen(rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip):
    return ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip,
        fill_mode='nearest'
    )

def residual_block(x, filters, kernel_size):
    shortcut = x
    x = Conv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('tanh')(x)
    x = Conv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
        
    x = Add()([x, shortcut])
    x = Activation('tanh')(x)
    return x

def build_complex_model(input_shape, num_residual_blocks, dropout_rate, learning_rate, filters, kernel_size, num_dense_layers, activation_function):
    # Input layer for combined images
    combined_input = Input(shape=(input_shape[1], input_shape[2], input_shape[3]), name='combined_input')
    
    # Convolutional base
    x = Conv2D(filters, (kernel_size, kernel_size), activation=activation_function, padding='same')(combined_input)
    x = MaxPooling2D((2, 2))(x)
    for _ in range(num_residual_blocks):
        x = residual_block(x, filters, kernel_size)
        if x.shape[1] >= 2 and x.shape[2] >= 2:
            x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    
    # Fully connected layers
    for _ in range(num_dense_layers):
        x = Dense(2048, activation=activation_function)(x)
        x = Dropout(dropout_rate)(x)
        x = BatchNormalization()(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = X_train.shape

# Define the specified parameters
num_residual_blocks = 9
dropout_rate = 0.35
learning_rate = 0.0005
filters = 128
kernel_size = 1
num_dense_layers = 1
activation_function = 'tanh'
rotation_range = 0
width_shift_range = 0.0
height_shift_range = 0.0
shear_range = 0.4
zoom_range = 0.0
horizontal_flip = False

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Callbacks for training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Custom callback to print epoch details
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}/{self.params['epochs']}: loss={logs['loss']:.4f}, accuracy={logs['accuracy']:.4f}, val_loss={logs['val_loss']:.4f}, val_accuracy={logs['val_accuracy']:.4f}")

# Create data generator
datagen = create_datagen(rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip)
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

# Build the model
model = build_complex_model(input_shape, num_residual_blocks, dropout_rate, learning_rate, filters, kernel_size, num_dense_layers, activation_function)

# Train the model
with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr, early_stopping, CustomCallback()],
        class_weight=class_weights,
        verbose=1
    )

Class weights: {0: 0.8640915593705293, 1: 1.18664047151277}


C:\Anaconda\envs\thesis\lib\site-packages\keras\preprocessing\image.py:2094: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(
C:\Anaconda\envs\thesis\lib\site-packages\keras\preprocessing\image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(


Epoch 1/50
604/604 [==============================] - 33s 41ms/step - loss: 1.1064 - accuracy: 0.5159 - val_loss: 1.1343 - val_accuracy: 0.4520 - lr: 5.0000e-04
Epoch 2/50
604/604 [==============================] - 24s 39ms/step - loss: 0.8343 - accuracy: 0.5350 - val_loss: 0.7431 - val_accuracy: 0.4884 - lr: 5.0000e-04
Epoch 3/50
604/604 [==============================] - 24s 39ms/step - loss: 0.7546 - accuracy: 0.5851 - val_loss: 0.8029 - val_accuracy: 0.4917 - lr: 5.0000e-04
Epoch 4/50
604/604 [==============================] - 24s 39ms/step - loss: 0.6965 - accuracy: 0.6244 - val_loss: 0.7482 - val_accuracy: 0.5886 - lr: 5.0000e-04
Epoch 5/50
604/604 [==============================] - 24s 39ms/step - loss: 0.6480 - accuracy: 0.6647 - val_loss: 0.6373 - val_accuracy: 0.6763 - lr: 5.0000e-04
Epoch 6/50
604/604 [==============================] - 23s 38ms/step - loss: 0.6168 - accuracy: 0.6908 - val_loss: 0.5523 - val_accuracy: 0.7235 - lr: 5.0000e-04
Epoch 7/50
604/604 [==============

# Re-Validation of best model

In [3]:
# Make predictions
with tf.device('/GPU:0'):
    val_predictions = model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
y_val_true = np.argmax(y_val, axis=1)
y_val_pred = np.argmax(val_predictions, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_val_true, y_val_pred)

print("Confusion Matrix:")
print(conf_matrix)

# Generate the classification report
class_report = classification_report(y_val_true, y_val_pred, target_names=categories)

print("Classification Report:")
print(class_report)

38/38 [==============================] - 1s 23ms/step
Confusion Matrix:
[[724   0]
 [  0 484]]
Classification Report:
                   precision    recall  f1-score   support

Healthy_augmented       1.00      1.00      1.00       724
Damaged_augmented       1.00      1.00      1.00       484

         accuracy                           1.00      1208
        macro avg       1.00      1.00      1.00      1208
     weighted avg       1.00      1.00      1.00      1208



# Save the best model v2 with 50 epochs

In [8]:
# Saving the best refined version 2
# Save the best model to a local directory
model_save_path = 'best_refined_model_v2_50Epochs.h5'
model.save(model_save_path)
print(f"Best model saved to {model_save_path}")

Best model saved to best_refined_model_v2_50Epochs.h5


# Load and predict the Validation Set

In [9]:
# Load the best model from the local directory
loaded_model = tf.keras.models.load_model(model_save_path)
print(f"Model loaded from {model_save_path}")

# Make predictions using the loaded model
with tf.device('/GPU:0'):
    loaded_val_predictions = loaded_model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
loaded_y_val_pred = np.argmax(loaded_val_predictions, axis=1)

# Generate the confusion matrix for the loaded model
loaded_conf_matrix = confusion_matrix(y_val_true, loaded_y_val_pred)

print("Confusion Matrix for loaded model:")
print(loaded_conf_matrix)

# Generate the classification report for the loaded model
loaded_class_report = classification_report(y_val_true, loaded_y_val_pred, target_names=categories)

print("Classification Report for loaded model:")
print(loaded_class_report)

Model loaded from best_refined_model_v2_50Epochs.h5
38/38 [==============================] - 1s 10ms/step
Confusion Matrix for loaded model:
[[722   2]
 [  0 484]]
Classification Report for loaded model:
                   precision    recall  f1-score   support

Healthy_augmented       1.00      1.00      1.00       724
Damaged_augmented       1.00      1.00      1.00       484

         accuracy                           1.00      1208
        macro avg       1.00      1.00      1.00      1208
     weighted avg       1.00      1.00      1.00      1208



# Initial Grid Search

In [1]:
# Define data augmentation
def create_datagen(rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip):
    return ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip,
        fill_mode='nearest'
    )

def residual_block(x, filters, kernel_size):
    shortcut = x
    x = Conv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('tanh')(x)
    x = Conv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
        
    x = Add()([x, shortcut])
    x = Activation('tanh')(x)
    return x

def build_complex_model(input_shape, num_residual_blocks, dropout_rate, learning_rate, filters, kernel_size, num_dense_layers, activation_function):
    # Input layer for combined images
    combined_input = Input(shape=(input_shape[1], input_shape[2], input_shape[3]), name='combined_input')
    
    # Convolutional base
    x = Conv2D(filters, (kernel_size, kernel_size), activation=activation_function, padding='same')(combined_input)
    x = MaxPooling2D((2, 2))(x)
    for _ in range(num_residual_blocks):
        x = residual_block(x, filters, kernel_size)
        if x.shape[1] >= 2 and x.shape[2] >= 2:
            x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    
    # Fully connected layers
    for _ in range(num_dense_layers):
        x = Dense(2048, activation=activation_function)(x)
        x = Dropout(dropout_rate)(x)
        x = BatchNormalization()(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = X_train.shape

# Define refined grid search parameters
num_residual_blocks_options = [7, 8, 9]
dropout_rate_options = [0.35, 0.4, 0.45]
learning_rate_options = [0.001, 0.0005, 0.0001]
filters_options = [128]
kernel_size_options = [1]
num_dense_layers_options = [1]
activation_function_options = ['tanh']
rotation_range_options = [0, 10, 20]
width_shift_range_options = [0.0, 0.1, 0.2]
height_shift_range_options = [0.0, 0.1, 0.2]
shear_range_options = [0.4, 0.5, 0.6]
zoom_range_options = [0.0, 0.1, 0.2]
horizontal_flip_options = [False, True]

# Create grid search parameter combinations
refined_parameter_combinations = list(product(num_residual_blocks_options, dropout_rate_options, learning_rate_options,
                                              filters_options, kernel_size_options, num_dense_layers_options,
                                              activation_function_options, rotation_range_options, width_shift_range_options,
                                              height_shift_range_options, shear_range_options, zoom_range_options, horizontal_flip_options))

# Select 50 unique combinations
np.random.seed(42)
selected_refined_combinations = np.random.choice(len(refined_parameter_combinations), 50, replace=False)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Callbacks for training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Custom callback to print epoch details
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}/{self.params['epochs']}: loss={logs['loss']:.4f}, accuracy={logs['accuracy']:.4f}, val_loss={logs['val_loss']:.4f}, val_accuracy={logs['val_accuracy']:.4f}")

# Perform refined grid search
best_val_accuracy_refined = 0
best_params_refined = None
best_model_save_path = 'best_refined_model.h5'

for idx, combination_idx in enumerate(selected_refined_combinations):
    num_residual_blocks, dropout_rate, learning_rate, filters, kernel_size, num_dense_layers, activation_function, rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip = refined_parameter_combinations[combination_idx]
    print(f"\nRefined Training Combination {idx + 1}/50: num_residual_blocks={num_residual_blocks}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, filters={filters}, kernel_size={kernel_size}, num_dense_layers={num_dense_layers}, activation_function={activation_function}, rotation_range={rotation_range}, width_shift_range={width_shift_range}, height_shift_range={height_shift_range}, shear_range={shear_range}, zoom_range={zoom_range}, horizontal_flip={horizontal_flip}")
    
    datagen = create_datagen(rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip)
    datagen.fit(X_train)
    train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
    
    model = build_complex_model(input_shape, num_residual_blocks, dropout_rate, learning_rate, filters, kernel_size, num_dense_layers, activation_function)
    
    with tf.device('/GPU:0'):
        history = model.fit(
            train_generator,
            steps_per_epoch=len(X_train) // BATCH_SIZE,
            epochs=40,
            validation_data=(X_val, y_val),
            callbacks=[reduce_lr, early_stopping, CustomCallback()],
            class_weight=class_weights,
            verbose=1
        )
    
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Validation accuracy: {val_accuracy}")
    
    if val_accuracy > best_val_accuracy_refined:
        best_val_accuracy_refined = val_accuracy
        best_params_refined = (num_residual_blocks, dropout_rate, learning_rate, filters, kernel_size, num_dense_layers, activation_function, rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip)
        model.save(best_model_save_path)
        print(f"Model with validation accuracy {val_accuracy} saved to {best_model_save_path}")

print(f"\nRefined best parameters found: num_residual_blocks={best_params_refined[0]}, dropout_rate={best_params_refined[1]}, learning_rate={best_params_refined[2]}, filters={best_params_refined[3]}, kernel_size={best_params_refined[4]}, num_dense_layers={best_params_refined[5]}, activation_function={best_params_refined[6]}, rotation_range={best_params_refined[7]}, width_shift_range={best_params_refined[8]}, height_shift_range={best_params_refined[9]}, shear_range={best_params_refined[10]}, zoom_range={best_params_refined[11]}, horizontal_flip={best_params_refined[12]}")
print(f"Refined best validation accuracy: {best_val_accuracy_refined}")

# Train the final model with the refined best parameters found
final_model = build_complex_model(input_shape, best_params_refined[0], best_params_refined[1], best_params_refined[2])

with tf.device('/GPU:0'):
    final_history = final_model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr, early_stopping, CustomCallback()],
        class_weight=class_weights,
        verbose=1
    )

# Save the best model to a local directory
model_save_path = 'best_model_best_acc.h5'
final_model.save(model_save_path)
print(f"Best model saved to {model_save_path}")

# Make predictions
with tf.device('/GPU:0'):
    val_predictions = final_model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
y_val_true = np.argmax(y_val, axis=1)
y_val_pred = np.argmax(val_predictions, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_val_true, y_val_pred)

print("Confusion Matrix:")
print(conf_matrix)

# Generate the classification report
class_report = classification_report(y_val_true, y_val_pred, target_names=categories)

print("Classification Report:")
print(class_report)

TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (4832, 64, 64, 6)
Validation data shape: (1208, 64, 64, 6)
Training labels shape: (4832, 2)
Validation labels shape: (1208, 2)
Class weights: {0: 0.8640915593705293, 1: 1.18664047151277}

Refined Training Combination 1/50: num_residual_blocks=9, dropout_rate=0.35, learning_rate=0.001, filters=128, kernel_size=1, num_dense_layers=1, activation_function=tanh, rotation_range=10, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.4, zoom_range=0.0, horizontal_flip=True


C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\keras\preprocessing\image.py:2094: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(
C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\keras\preprocessing\image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(


Epoch 1/40
604/604 [==============================] - 13s 16ms/step - loss: 1.1164 - accuracy: 0.5099 - val_loss: 1.1476 - val_accuracy: 0.3998 - lr: 0.0010
Epoch 2/40
604/604 [==============================] - 9s 15ms/step - loss: 0.8680 - accuracy: 0.5141 - val_loss: 0.8804 - val_accuracy: 0.4776 - lr: 0.0010
Epoch 3/40
604/604 [==============================] - 9s 15ms/step - loss: 0.8590 - accuracy: 0.5135 - val_loss: 0.7617 - val_accuracy: 0.5919 - lr: 0.0010
Epoch 4/40
604/604 [==============================] - 9s 16ms/step - loss: 0.7965 - accuracy: 0.5348 - val_loss: 0.6949 - val_accuracy: 0.5985 - lr: 0.0010
Epoch 5/40
604/604 [==============================] - 9s 15ms/step - loss: 0.8020 - accuracy: 0.5408 - val_loss: 0.7472 - val_accuracy: 0.5604 - lr: 0.0010
Epoch 6/40
604/604 [==============================] - 9s 15ms/step - loss: 0.7263 - accuracy: 0.6113 - val_loss: 0.6807 - val_accuracy: 0.5944 - lr: 0.0010
Epoch 7/40
604/604 [==============================] - 9s 15ms/s

TypeError: build_complex_model() missing 4 required positional arguments: 'filters', 'kernel_size', 'num_dense_layers', and 'activation_function'

# Loaded Model Validation

In [9]:
# Load the best model from the local directory
model_save_path = 'best_refined_model.h5'
loaded_model = tf.keras.models.load_model(model_save_path)
print(f"Model loaded from {model_save_path}")

# Make predictions using the loaded model
with tf.device('/GPU:0'):
    loaded_val_predictions = loaded_model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
y_val_true = np.argmax(y_val, axis=1)
y_val_pred = np.argmax(loaded_val_predictions, axis=1)

# Convert one-hot encoded predictions and true labels to label indices
loaded_y_val_pred = np.argmax(loaded_val_predictions, axis=1)

# Generate the confusion matrix for the loaded model
loaded_conf_matrix = confusion_matrix(y_val_true, loaded_y_val_pred)

print("Confusion Matrix for loaded model:")
print(loaded_conf_matrix)

# Generate the classification report for the loaded model
loaded_class_report = classification_report(y_val_true, loaded_y_val_pred, target_names=categories)

print("Classification Report for loaded model:")
print(loaded_class_report)

Model loaded from best_refined_model.h5
38/38 [==============================] - 1s 11ms/step
Confusion Matrix for loaded model:
[[712  12]
 [  3 481]]
Classification Report for loaded model:
                   precision    recall  f1-score   support

Healthy_augmented       1.00      0.98      0.99       724
Damaged_augmented       0.98      0.99      0.98       484

         accuracy                           0.99      1208
        macro avg       0.99      0.99      0.99      1208
     weighted avg       0.99      0.99      0.99      1208

